In [1]:
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from scipy import misc as scipyMisc

import os

import PIL.Image
from keras.models import load_model
import numpy as np
from skimage.transform import resize
import datetime
import time


/var/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# URL for GET- or POST-Request
url = 'https://phinau.de/trasi'

# API-Key
access_key = 'ehiefoveingereim3ooD2vo8reeb9ooz'

# Key-Value pair: API-Key
credentials = {'key': access_key}


In [3]:
#####################################################################
def load_ppm_image(path):
    i = mpimg.imread(path)
    a = scipyMisc.imresize(i, (64,64,3), interp='bicubic', mode=None)
    return a

#parsed = load_ppm_image(filename)
#plt.imshow(parsed)
#plt.show()
#####################################################################

In [4]:
def initialize_model(filepath):
    model = load_model(filepath)
    #model.summary()
    return model


def load_image(file_path):
    im = PIL.Image.open(file_path)
    im = np.asarray(im)
    return im - 127.5


def predict_single_image_from_model(model, image_filepath, shape):
    loaded_png_image = load_image(image_filepath)
    resized_png_image = resize(loaded_png_image, (shape, shape))
    x = np.expand_dims(resized_png_image, axis=0)
    preds = model.predict(x)
    label = np.argmax(preds)
    image_score = predict_single_image(model, resized_png_image)[0]
    image_label_score = image_score[label]
    return image_label_score


def predict_single_image(model,img):
    imgArr = (np.expand_dims(img,0))
    imgArr = imgArr/255
    return model.predict(imgArr)[0],img

In [5]:
def submit_single_image(filepath):

    filename = filepath
    file_to_upload = {'image': open(filename,'rb')}

    pimg_bytes = open(filename,'rb')
    file_to_upload = {'image': pimg_bytes}

    post_request = requests.post(url, files=file_to_upload, data=credentials)

    if post_request.status_code == 200:
        post = post_request.text
        # post_request.text # For Debugging

        trasi_score = json.loads(post)

        _class = trasi_score[0].get('class')
        _confidence = trasi_score[0].get('confidence')
        trasi_resp = "Filename:" + filename + ";Class:" + str(_class) + ";Confidence:" + str(_confidence) + "\n"
        print(trasi_resp)

    else:
        print("HTTP-Statuscode: " + post_request.status_code)
    
    return trasi_score


def submit_images_from_directory(input_folder, model_filepath, shape, min_confidence):
   
    with open("######Trasi-Submission-results_" + method + "_" + underlying_model + ".txt", "a") as myfile:
        myfile.write("DateTime;Filename;Trasi-Class;Trasi-Confidence" + "\n")
            
    with open("######Trasi-Submission-results-greater-than-" + str(min_confidence) + "_" + method + "_" + underlying_model + ".txt", "a") as myfile:
        myfile.write("DateTime;Filename;Trasi-Class;Trasi-Confidence" + "\n")
        
    #with open("######Trasi-And-Local-results_" + method + "_" + underlying_model + ".txt", "a") as myfile:
    #    myfile.write("DateTime;Filename;Trasi-Class;Trasi-Confidence;Local-Confidence" + "\n")

    counter = 0
        
    for file in os.listdir(input_folder):
        
        if counter % 60 == 0:
            print("Time to sleep for 61 sec...")
            time.sleep(61)
            
        filename = os.path.join(input_folder, file)

        file_to_upload = {'image': open(filename,'rb')}
        
        post_request = requests.post(url, files=file_to_upload, data=credentials)

        if post_request.status_code == 200:
            post = post_request.text
            # post_request.text # For Debugging

            trasi_score = json.loads(post)

            _class = trasi_score[0].get('class')
            _confidence = trasi_score[0].get('confidence')
            trasi_resp = filename + ";" + str(_class) + ";" + str(_confidence)
            #print(trasi_resp) For Debugging
            
            with open("######Trasi-Submission-results_" + method + "_" + underlying_model + ".txt", "a") as myfile:
                myfile.write(str(datetime.datetime.now()) + ";" + trasi_resp + "\n")
            
            if _confidence >= 0.90:
                with open("######Trasi-Submission-results-greater-than-" + str(min_confidence) + "_" + method + "_" + underlying_model + ".txt", "a") as myfile:
                    myfile.write(str(datetime.datetime.now()) + ";" + trasi_resp + "\n")

            #initialized_model = initialize_model(model_filepath)
            #local_prediction = predict_single_image_from_model(initialized_model, filename, shape)
            
            #with open("######Trasi-And-Local-results_" + method + "_" + underlying_model + ".txt", "a") as myfile:
            #    myfile.write(str(datetime.datetime.now()) + ";" + trasi_resp + ";" + str(local_prediction))
   
        else:
            print("HTTP-Statuscode: " + str(post_request.status_code))
        
        counter += 1
        print("Submitted file: " + str(counter) + "/" + str(len(os.listdir(input_folder))))
        
    print("\nEND of Submission.")

    return trasi_score


In [6]:
#filepath = '../pytorch-alexnet-gtsrb/generated/Target_Confidence_100/ga_fooling_class_1.png'
#result = submit_single_image(filepath)

In [8]:
method = "GA_100"
underlying_model = "Alexnet"
input_folder = "generated/Target_Confidence_100/"
shape = 64
min_confidence = 0.90

submit_images_from_directory(input_folder, "", shape, min_confidence)

Time to sleep for 61 sec...
Submitted file: 1/43
Submitted file: 2/43
Submitted file: 3/43
Submitted file: 4/43
Submitted file: 5/43
Submitted file: 6/43
Submitted file: 7/43
Submitted file: 8/43
Submitted file: 9/43
Submitted file: 10/43
Submitted file: 11/43
Submitted file: 12/43
Submitted file: 13/43
Submitted file: 14/43
Submitted file: 15/43
Submitted file: 16/43
Submitted file: 17/43
Submitted file: 18/43
Submitted file: 19/43
Submitted file: 20/43
Submitted file: 21/43
Submitted file: 22/43
Submitted file: 23/43
Submitted file: 24/43
Submitted file: 25/43
Submitted file: 26/43
Submitted file: 27/43
Submitted file: 28/43
Submitted file: 29/43
Submitted file: 30/43
Submitted file: 31/43
Submitted file: 32/43
Submitted file: 33/43
Submitted file: 34/43
Submitted file: 35/43
Submitted file: 36/43
Submitted file: 37/43
Submitted file: 38/43
Submitted file: 39/43
Submitted file: 40/43
Submitted file: 41/43
Submitted file: 42/43
Submitted file: 43/43

END of Submission.


[{'class': 'Gefahrenstelle', 'confidence': 0.23941961},
 {'class': 'Ende des Überholverbotes für Kraftfahrzeuge mit einer zulässigen Gesamtmasse über 3,5t',
  'confidence': 0.01768792},
 {'class': 'Kurve (links)', 'confidence': 0.01768792},
 {'class': 'Verbot der Einfahrt', 'confidence': 0.01768792},
 {'class': 'Verbot für Kraftfahrzeuge mit einer zulässigen Gesamtmasse von 3,5t',
  'confidence': 0.01768792}]

In [9]:
print("END Gradient Ascent TC 100 " + underlying_model)

END Gradient Ascent TC 100 Alexnet


In [10]:
method = "GA_99"
underlying_model = "Alexnet"
input_folder = "generated/Target_Confidence_99/"
shape = 64
min_confidence = 0.90

submit_images_from_directory(input_folder, "", shape, min_confidence)

Time to sleep for 61 sec...
Submitted file: 1/43
Submitted file: 2/43
Submitted file: 3/43
Submitted file: 4/43
Submitted file: 5/43
Submitted file: 6/43
Submitted file: 7/43
Submitted file: 8/43
Submitted file: 9/43
Submitted file: 10/43
Submitted file: 11/43
Submitted file: 12/43
Submitted file: 13/43
Submitted file: 14/43
Submitted file: 15/43
Submitted file: 16/43
Submitted file: 17/43
Submitted file: 18/43
Submitted file: 19/43
Submitted file: 20/43
Submitted file: 21/43
Submitted file: 22/43
Submitted file: 23/43
Submitted file: 24/43
Submitted file: 25/43
Submitted file: 26/43
Submitted file: 27/43
Submitted file: 28/43
Submitted file: 29/43
Submitted file: 30/43
Submitted file: 31/43
Submitted file: 32/43
Submitted file: 33/43
Submitted file: 34/43
Submitted file: 35/43
Submitted file: 36/43
Submitted file: 37/43
Submitted file: 38/43
Submitted file: 39/43
Submitted file: 40/43
Submitted file: 41/43
Submitted file: 42/43
Submitted file: 43/43

END of Submission.


[{'class': 'Gefahrenstelle', 'confidence': 0.08031172},
 {'class': 'Wildwechsel', 'confidence': 0.06988996},
 {'class': 'Baustelle', 'confidence': 0.03214405},
 {'class': 'Rechts vorbei', 'confidence': 0.02420756},
 {'class': 'Verbot der Einfahrt', 'confidence': 0.01983617}]

In [11]:
print("END Gradient Ascent TC 99 " + underlying_model)

END Gradient Ascent TC 99 Alexnet
